In [ ]:
##### CREATING INVERTED INDEX FROM TOKENIZED TEXTS #####

In [ ]:
### conversion from strings to tokens within lists ###
def Convert(string):
    li = list(string.split(" "))
    return li

In [ ]:
### function to create an inverted index ###
from collections import defaultdict
def create_index (data):
    index = defaultdict(list)
    for i, tokens in enumerate(data):
        for token in tokens:
             index[token].append(i)
    return index

In [ ]:
### data processing ###
import html
import os, sys
from lxml import etree
import xml.etree.ElementTree as ET

# initialization
kolekce = []
hashed = []
text_ready = []

doc_ID_dict = {} # dict -> order of doc processed as key, DOCID as value

# counters 
counter_child = 0
counter_node = 0
counter_doc_ID = 0
counter_file = 0

#path from where to load xml files
path = "/home/mazurri/InfoRetrieval"
directory = os.listdir(path)

for file in directory:
    counter_child = 0
    counter_file = counter_file + 1
    test = os.path.splitext(file)
    if test[0] == "queries_cs":  # queries_cs is a xml file for the second part of the task -> queries
        continue      
    elif test[1] == ".xml":
        print(file)
        #kvůli erroru s unescaped characters "&"
        parser = etree.XMLParser(recover=True)
        tree = ET.parse(file, parser=parser)
        root = tree.getroot()
        for child in root:
            
            ### populating doc_ID_dict
            doc_ID = child.find("DOCID").text
            doc_ID_dict[counter_doc_ID] = doc_ID
            counter_doc_ID = counter_doc_ID + 1
            
            for node in child:
                if counter_node < 3: # first 3 nodes do not contain text to be processed
                    counter_node = counter_node + 1
                    continue
                else:
                    text = root[counter_child][counter_node].text
                    
                    counter_node = counter_node + 1

                    ### normalization of texts ###
                    text = text.lower()
                    text = text.replace("\n", " ")
                    text = text.replace("?", " ")
                    text = text.replace("!", " ")
                    text = text.replace(".", " ")
                    text = text.replace(",", " ")
                    text = text.replace(" - ", " ")
                    
                    # tokenize as per desired format
                    text = Convert(text)
                    
                    ### hashing the result tokens###
                    for i in text:
                        hashed.append(hash(i))
                    
            ### eliminating duplicates ###
            hashed_set = set(hashed)
            hashed = []

            ### making a final set of tokens ###
            kolekce.append(hashed_set)
            hashed_set = None
  
            counter_node = 0
            counter_child = counter_child + 1
    else:
        continue

In [ ]:
### using inverted index to match tokens and counter_doc_ID ###
inverted_index = create_index(kolekce)

In [ ]:
##### CREATING CUSTOM QUERY PARSER AND SEARCH ENGINE ######

In [ ]:
### INPUT ###

In [ ]:
operatory = ["AND", "OR", "AND_NOT"]

In [ ]:
def findWithinIndex (data):
    data_new = data
    token_index = 0
    for i in data_new:
        text = None
        if i in operatory:
            continue
        else:
            data_new[token_index] = inverted_index[hash(i.lower())]
        token_index = token_index + 1
    return data_new

In [ ]:
def better_findDocuments (data):
    query_docs = []
    query_docs_temp = []
    for token_index, token in enumerate(data):
        if token_index == 0:
            for item in token:
                query_docs.append(item)
        elif token_index % 2 !=0:
            
            next_token = data[token_index+1]
       
            short_list = min(query_docs, next_token, key=len)
            long_list = max(query_docs, next_token, key=len)
            
            if token == "OR":
                query_docs = sorted(list(set(query_docs+next_token))) #připoj co následuje za OR do query_docs
            
            if token == "AND":
                query_docs = sorted(list(set(short_list).intersection(set(long_list))))
            
            if token == "AND_NOT":
                query_docs = sorted(list(set(short_list)-set(long_list)))               
                
        else:
            continue
    return query_docs

In [ ]:
import re

counter_query_ID = 0

operatory = ["AND", "OR", "AND_NOT"]

for file in directory:
    test = os.path.splitext(file)
    if test[0] == "queries_cs":
        parser = etree.XMLParser(recover=True)
        tree = ET.parse(file, parser = parser)
        root = tree.getroot()
        
        for counter_query_ID, child in enumerate(root):
            doc_NUM = child.find("num").text
            query = child.find("query").text            
            query = re.sub("AND NOT", "AND_NOT", query)

            converted_query = Convert(query)

            if "" in converted_query:
                converted_query.remove("")
                
            hashed_query = findWithinIndex(converted_query)
            vysledek =  better_findDocuments(hashed_query)
            
            new_list = []
            for item in vysledek:
                if item > len(doc_ID_dict):
                    new_list.append(item)
                else:
                    new_list.append(doc_ID_dict[item])
            
            full_str = ' '.join([str(elem) for elem in new_list])

            # creating result file
            with open("vysledky.txt", "a+") as final_file:
                final_file.seek(0)
                data = final_file.read(100)
                if len(data) > 0 :
                    final_file.write("\n")
                # Append text at the end of file
                final_file.write("\n")
                final_file.write("\n")
                final_file.write(doc_NUM)
                final_file.write("\n")
                final_file.write(full_str)

            new_list = []

    else:
        continue